LIBRARIES NEEDED

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

LOADING CSV FILES USING pandas

In [75]:
df_train=pd.read_csv("/content/train_0OECtn8.csv")#training data as df_train
df_test=pd.read_csv("/content/test_1zqHu22.csv")#test data as df_test

In [76]:
print(df_train.shape)#shape of training set
print(df_test.shape)#shape of test set

(89197, 10)
(11121, 9)


In [77]:
print(df_train.isnull().sum())#checking for null values in the training set
print(df_test.isnull().sum())#checking for null values in the test set

row_id              0
user_id             0
category_id         0
video_id            0
age                 0
gender              0
profession          0
followers           0
views               0
engagement_score    0
dtype: int64
row_id         0
user_id        0
category_id    0
video_id       0
age            0
gender         0
profession     0
followers      0
views          0
dtype: int64


In [78]:
print(df_train.dtypes)#datatypes of columns in the training set
print(df_test.dtypes)#datatypes of columns in the test set

row_id                int64
user_id               int64
category_id           int64
video_id              int64
age                   int64
gender               object
profession           object
followers             int64
views                 int64
engagement_score    float64
dtype: object
row_id          int64
user_id         int64
category_id     int64
video_id        int64
age             int64
gender         object
profession     object
followers       int64
views           int64
dtype: object


In [79]:
print(df_train.gender.value_counts())#count for each unique value in gender columns of train data
print(df_train.profession.value_counts())#count for each unique value in gender columns of train data
print(df_train.user_id.value_counts())#count for each unique value in gender columns of train data

Male      52397
Female    36800
Name: gender, dtype: int64
Student                 44638
Other                   26840
Working Professional    17719
Name: profession, dtype: int64
5198     10
13218    10
1448     10
13410    10
7157     10
         ..
21507     2
4404      2
21430     2
17541     2
11279     2
Name: user_id, Length: 27734, dtype: int64


In [80]:
df_train.head()#printing first 5 rows of the training data

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35
3,4,12597,23,112,19,Male,Student,220,613,3.77
4,5,13626,23,112,27,Male,Working Professional,220,613,3.13


In [81]:
df_train.corr()#to analyse the correlation between different columns in the data

,row_id,user_id,category_id,video_id,age,followers,views,engagement_score
row_id,1.000000,-0.006953,0.002833,0.000891,-0.005199,-0.001374,-0.001815,-0.003368
user_id,-0.006953,1.000000,0.002139,0.005232,-0.000036,-0.002026,-0.001843,-0.002900
category_id,0.002833,0.002139,1.000000,0.556550,-0.003152,0.056432,0.079321,-0.094288
video_id,0.000891,0.005232,0.556550,1.000000,-0.005615,0.098778,-0.029448,-0.030552
age,-0.005199,-0.000036,-0.003152,-0.005615,1.000000,0.004623,0.011042,-0.232626
followers,-0.001374,-0.002026,0.056432,0.098778,0.004623,1.000000,0.360808,0.000855
views,-0.001815,-0.001843,0.079321,-0.029448,0.011042,0.360808,1.000000,-0.084679
engagement_score,-0.003368,-0.002900,-0.094288,-0.030552,-0.232626,0.000855,-0.084679,1.000000


SPLITTING X(independent varible) AND Y(dependant variable) 

In [82]:
X=df_train.loc[:,"user_id":"views"]
print(X.head())
Y=df_train.iloc[:,-1]
print(Y.head())

   user_id  category_id  video_id  ...            profession followers views
0    19990           37       128  ...               Student       180  1000
1     5304           32       132  ...               Student       330   714
2     1840           12        24  ...               Student       180   138
3    12597           23       112  ...               Student       220   613
4    13626           23       112  ...  Working Professional       220   613

[5 rows x 8 columns]
0    4.33
1    1.79
2    4.35
3    3.77
4    3.13
Name: engagement_score, dtype: float64


In [83]:
object_cols=[cols for cols in X.columns if X[cols].dtype=="object"]#getting the names of categorical features in the data
print(object_cols)

['gender', 'profession']


In [84]:
"""encoded_X_train=X_train.copy()
encoded_X_valid=X_valid.copy()
ordinal_encoder = OrdinalEncoder()
encoded_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
encoded_X_valid[object_cols] = ordinal_encoder.fit_transform(X_valid[object_cols])
print(encoded_X_train.head)"""

'encoded_X_train=X_train.copy()\nencoded_X_valid=X_valid.copy()\nordinal_encoder = OrdinalEncoder()\nencoded_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])\nencoded_X_valid[object_cols] = ordinal_encoder.fit_transform(X_valid[object_cols])\nprint(encoded_X_train.head)'

In [85]:
hot=OneHotEncoder(handle_unknown="ignore",sparse=False)    #one hot encoding
hot_X=pd.DataFrame(hot.fit_transform(X[object_cols]))  #encoding training data
hot_X.index=X.index    #encoded dataset loses index so getting it back
hot_X.columns=hot.get_feature_names(["gender","profession"])   #getting column names for newly encoded columns
X.drop(object_cols,axis=1,inplace=True)  #dropping unwanted columns
finalhot_X=pd.concat([X,hot_X],axis=1)   #joining normal and encoded data
print(finalhot_X.shape)   #shape of the encoded data

(89197, 11)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [86]:
finalhot_X.head()#first 5 rows of the encoded data

,user_id,category_id,video_id,age,followers,views,gender_Female,gender_Male,profession_Other,profession_Student,profession_Working Professional
0,19990,37,128,24,180,1000,0.0,1.0,0.0,1.0,0.0
1,5304,32,132,14,330,714,1.0,0.0,0.0,1.0,0.0
2,1840,12,24,19,180,138,0.0,1.0,0.0,1.0,0.0
3,12597,23,112,19,220,613,0.0,1.0,0.0,1.0,0.0
4,13626,23,112,27,220,613,0.0,1.0,0.0,0.0,1.0


In [87]:
my_model=RandomForestRegressor(n_estimators=500,n_jobs=10,random_state=20,verbose=False) #creating random forest regressor model
my_model.fit(finalhot_X,Y) #fitting the model with X and Y

RandomForestRegressor(n_estimators=500, n_jobs=10, random_state=20,
                      verbose=False)

PREPARING TEST DATA FOR PREDICTIONS

In [88]:
df_test.head()#printing first 5 rows of the test data

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views
0,89198,7986,12,42,14,Male,Student,180,138
1,89199,11278,34,115,14,Male,Student,230,840
2,89200,17245,8,110,44,Female,Working Professional,280,628
3,89201,9851,16,137,18,Male,Student,270,462
4,89202,16008,34,96,47,Female,Other,230,840


In [89]:
X_test=df_test.loc[:,"user_id":"views"]#extracting the columns needed
print(X_test.head())
hot_X_test=pd.DataFrame(hot.transform(X_test[object_cols]))#one hot encoding test data
hot_X_test.index=X_test.index
X_test.drop(object_cols,axis=1,inplace=True)
finalhot_X_test=pd.concat([X_test,hot_X_test],axis=1)  # joing normal and encoded data
test_pred=my_model.predict(finalhot_X_test)  #predicting value for test data

   user_id  category_id  video_id  ...            profession followers views
0     7986           12        42  ...               Student       180   138
1    11278           34       115  ...               Student       230   840
2    17245            8       110  ...  Working Professional       280   628
3     9851           16       137  ...               Student       270   462
4    16008           34        96  ...                 Other       230   840

[5 rows x 8 columns]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


SAVING OUTPUT TO A CSV FILE

In [90]:
output = pd.DataFrame({'row_id': df_test.row_id,
                       'engagement_score': test_pred})
print(output)

       row_id  engagement_score
0       89198           4.04324
1       89199           3.12706
2       89200           2.67084
3       89201           3.74138
4       89202           1.72360
...       ...               ...
11116  100314           3.11116
11117  100315           3.16442
11118  100316           3.42738
11119  100317           3.86916
11120  100318           3.09644

[11121 rows x 2 columns]


In [91]:
output.to_csv('submission.csv', index=False)# saving as submission.csv file